In [26]:
clear_output()
!pip install imutils
!pip install tqdm
!conda install keras -y



import os
import boto3
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import time
import json
import shutil
from IPython.display import clear_output
import numpy as np 
from tqdm import tqdm
import cv2
import os
import shutil
import itertools
import imutils
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
import sagemaker.amazon.common as smac
from sagemaker import get_execution_role



role = get_execution_role()

bucket = 'neuroati-machine-learning'# enter your s3 bucket where you will copy data and model artifacts
prefix = 'brain-tumor/dataset'
dataset_file = 'brain-mri-images-for-brain-tumor-detection.zip'

s3 = boto3.client('s3')
!rm -rf 'dataset' TRAIN TEST VAL TRAIN_CROP TEST_CROP VAL_CROP keras-pretrained
!mkdir 'dataset'
s3.download_file(bucket, os.path.join(prefix, dataset_file), os.path.join('dataset', dataset_file))

!unzip 'dataset/brain-mri-images-for-brain-tumor-detection.zip' -d 'dataset/'
!rm 'dataset/brain-mri-images-for-brain-tumor-detection.zip'

!mkdir TRAIN TEST VAL TRAIN/YES TRAIN/NO TEST/YES TEST/NO VAL/YES VAL/NO

IMG_PATH = './dataset/brain_tumor_dataset/'
# split the data by train/val/test
for CLASS in os.listdir(IMG_PATH):
    if not CLASS.startswith('.'):
        IMG_NUM = len(os.listdir(IMG_PATH + CLASS))
        for (n, FILE_NAME) in enumerate(os.listdir(IMG_PATH + CLASS)):
            img = IMG_PATH + CLASS + '/' + FILE_NAME
            if n < 5:
                shutil.copy(img, 'TEST/' + CLASS.upper() + '/' + FILE_NAME)
            elif n < 0.8*IMG_NUM:
                shutil.copy(img, 'TRAIN/'+ CLASS.upper() + '/' + FILE_NAME)
            else:
                shutil.copy(img, 'VAL/'+ CLASS.upper() + '/' + FILE_NAME)

                
def load_data(dir_path, img_size=(100,100)):
    """
    Load resized images as np.arrays to workspace
    """
    X = []
    y = []
    i = 0
    labels = dict()
    for path in tqdm(sorted(os.listdir(dir_path))):
        if not path.startswith('.'):
            labels[i] = path
            for file in os.listdir(dir_path + path):
                if not file.startswith('.'):
                    img = cv2.imread(dir_path + path + '/' + file)
                    X.append(img)
                    y.append(i)
            i += 1
    X = np.array(X)
    y = np.array(y)
    print(f'{len(X)} images loaded from {dir_path} directory.')
    return X, y, labels

TRAIN_DIR = 'TRAIN/'
TEST_DIR = 'TEST/'
VAL_DIR = 'VAL/'
IMG_SIZE = (224,224)

# use predefined function to load the image data into workspace
X_train, y_train, labels = load_data(TRAIN_DIR, IMG_SIZE)
X_test, y_test, _ = load_data(TEST_DIR, IMG_SIZE)
X_val, y_val, _ = load_data(VAL_DIR, IMG_SIZE)

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Archive:  dataset/brain-mri-images-for-brain-tumor-detection.zip
   creating: dataset/brain_tumor_dataset/
   creating: dataset/brain_tumor_dataset/no/
  inflating: dataset/brain_tumor_dataset/no/30 no.jpg  
  inflating: dataset/brain_tumor_dataset/no/22 no.jpg  
  inflating: dataset/brain_tumor_dataset/no/41 no.jpg  
  inflating: dataset/brain_tumor_dataset/no/14 no.jpg  
  inflating: dataset/brain_tumor_dataset/no/no 10.jpg  
  inflating: dataset/

  inflating: dataset/brain_tumor_dataset/yes/Y146.JPG  
  inflating: dataset/brain_tumor_dataset/yes/Y185.jpg  
  inflating: dataset/brain_tumor_dataset/yes/Y81.jpg  
  inflating: dataset/brain_tumor_dataset/yes/Y95.jpg  
  inflating: dataset/brain_tumor_dataset/yes/Y42.jpg  
  inflating: dataset/brain_tumor_dataset/yes/Y56.jpg  
  inflating: dataset/brain_tumor_dataset/yes/Y184.JPG  
  inflating: dataset/brain_tumor_dataset/yes/Y2.jpg  
  inflating: dataset/brain_tumor_dataset/yes/Y147.JPG  
  inflating: dataset/brain_tumor_dataset/yes/Y153.jpg  
  inflating: dataset/brain_tumor_dataset/yes/Y108.jpg  
  inflating: dataset/brain_tumor_dataset/yes/Y120.JPG  
  inflating: dataset/brain_tumor_dataset/yes/Y242.JPG  
  inflating: dataset/brain_tumor_dataset/yes/Y256.JPG  
  inflating: dataset/brain_tumor_dataset/yes/Y18.JPG  
  inflating: dataset/brain_tumor_dataset/yes/Y24.jpg  
 extracting: dataset/brain_tumor_dataset/yes/Y30.jpg  
  inflating: dataset/brain_tumor_dataset/yes/Y31.jpg  
  

100%|██████████| 2/2 [00:00<00:00, 12.96it/s]

193 images loaded from TRAIN/ directory.
10 images loaded from TEST/ directory.
50 images loaded from VAL/ directory.


In [27]:
def crop_imgs(set_name, add_pixels_value=0):
    """
    Finds the extreme points on the image and crops the rectangular out of them
    """
    set_new = []
    for img in set_name:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        gray = cv2.GaussianBlur(gray, (5, 5), 0)

        # threshold the image, then perform a series of erosions +
        # dilations to remove any small regions of noise
        thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.erode(thresh, None, iterations=2)
        thresh = cv2.dilate(thresh, None, iterations=2)

        # find contours in thresholded image, then grab the largest one
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        c = max(cnts, key=cv2.contourArea)

        # find the extreme points
        extLeft = tuple(c[c[:, :, 0].argmin()][0])
        extRight = tuple(c[c[:, :, 0].argmax()][0])
        extTop = tuple(c[c[:, :, 1].argmin()][0])
        extBot = tuple(c[c[:, :, 1].argmax()][0])

        ADD_PIXELS = add_pixels_value
        new_img = img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()
        set_new.append(new_img)

    return np.array(set_new)

def save_new_images(x_set, y_set, folder_name):
    i = 0
    for (img, imclass) in zip(x_set, y_set):
        if imclass == 0:
            cv2.imwrite(folder_name+'NO/'+str(i)+'.jpg', img)
        else:
            cv2.imwrite(folder_name+'YES/'+str(i)+'.jpg', img)
        i += 1

def preprocess_imgs(set_name, img_size):
    """
    Resize and apply VGG-15 preprocessing
    """
    set_new = []
    for img in set_name:
        img = cv2.resize(
            img,
            dsize=img_size,
            interpolation=cv2.INTER_CUBIC
        )
        set_new.append(preprocess_input(img))
    return np.array(set_new)

X_train_crop = crop_imgs(set_name=X_train)
X_val_crop = crop_imgs(set_name=X_val)
X_test_crop = crop_imgs(set_name=X_test)

# saving new images to the folder
!rm -rf TRAIN_CROP TEST_CROP VAL_CROP TRAIN_CROP/YES TRAIN_CROP/NO TEST_CROP/YES TEST_CROP/NO VAL_CROP/YES VAL_CROP/NO
!mkdir TRAIN_CROP TEST_CROP VAL_CROP TRAIN_CROP/YES TRAIN_CROP/NO TEST_CROP/YES TEST_CROP/NO VAL_CROP/YES VAL_CROP/NO

save_new_images(X_train_crop, y_train, folder_name='TRAIN_CROP/')
save_new_images(X_val_crop, y_val, folder_name='VAL_CROP/')
save_new_images(X_test_crop, y_test, folder_name='TEST_CROP/')


Found 193 images belonging to 2 classes.
Found 50 images belonging to 2 classes.
